In [ ]:
!pip install git+https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/@Hai  
#my github for extra pos and lemma analysis

import convokit
from convokit import Corpus, download

corpus = Corpus(download('subreddit-Cornell')) 
!python -m spacy download en_core_web_sm


  Cloning https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/ (to revision Hai) to /tmp/pip-req-build-jmkq9asr
  Running command git clone -q https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/ /tmp/pip-req-build-jmkq9asr
  Running command git checkout -b Hai --track origin/Hai
  Switched to a new branch 'Hai'
  Branch 'Hai' set up to track remote branch 'Hai' from 'origin'.
     |████████████████████████████████| 12.7MB 332kB/s 
     |████████████████████████████████| 1.4MB 49.3MB/s 
     |████████████████████████████████| 245kB 34.8MB/s 
     |████████████████████████████████| 1.1MB 52.4MB/s 
     |████████████████████████████████| 9.1MB 47.9MB/s 
     |████████████████████████████████| 460kB 42.5MB/s 
     |████████████████████████████████| 133kB 40.1MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 122kB 49.5MB/s 
  Created wheel for convokit: filename=convokit-2.4.4-cp37-none-

In [ ]:
from convokit.text_processing import TextParser  
textparser = TextParser()
textparser.transform(corpus)     #perform SpaCy analysis

In [ ]:
#check use of PoS
import collections
convo = corpus.random_conversation() #pick random conversation from corpus 
d = collections.defaultdict(list)

d2 = collections.defaultdict(list)
for spkr in convo.iter_speakers():
   spkr_utts = list(spkr.iter_utterances())
   for i in spkr_utts:
      d[spkr.id].append(len(i.text.split(" "))) 
      for j in i.meta["parsed"]: 
        d2[spkr.id].append(list())
        for k in range(len(j["toks"])) :
          d2[spkr.id][-1].append(j["toks"][k]["pos"]) #find "parsed" pos metadata for each speaker (separate by utt)


for i in range(len(d2[spkr.id])):
    d2[spkr.id][i] = collections.Counter(d2[spkr.id][i])


#d2 gives parts of speech for each utterance and each speaker)

print(d2)

defaultdict(<class 'list'>, {'Kozinskey': [['PRON', 'VERB', 'PUNCT'], ['DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN', 'AUX', 'SCONJ', 'PRON', 'AUX', 'VERB', 'NOUN', 'SCONJ', 'PRON', 'ADV', 'VERB', 'PRON', 'ADP', 'ADJ', 'ADP', 'PUNCT', 'CCONJ', 'SCONJ', 'PRON', 'AUX', 'VERB', 'ADJ', 'NOUN', 'DET', 'AUX', 'ADV', 'VERB', 'ADP', 'NOUN', 'NOUN', 'PUNCT'], ['PRON', 'VERB', 'ADP', 'NOUN', 'ADP', 'ADJ', 'ADP', 'PRON', 'NOUN', 'NOUN', 'SCONJ', 'VERB', 'ADJ', 'CCONJ', 'PRON', 'VERB', 'DET', 'ADP', 'VERB', 'ADP', 'DET', 'NOUN', 'NOUN', 'PUNCT'], ['ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'DET', 'ADJ', 'NOUN', 'SCONJ', 'SCONJ', 'PRON', 'VERB', 'PART', 'ADP', 'DET', 'NOUN', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'ADV', 'VERB', 'DET', 'NOUN', 'NOUN', 'PUNCT'], ['PRON', 'AUX', 'ADV', 'ADJ', 'ADP', 'PRON', 'NOUN', 'PUNCT'], ['VERB', 'PART', 'VERB', 'ADV', 'PUNCT', 'CCONJ', 'VERB', 'NOUN', 'ADP', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PUNCT', 'PUNCT'], ['PRON', 'AUX', 'PART', 'VERB', 'ADP', 'NOUN', 'PUNC

In [ ]:
#check length of utterances
import pandas as pd

df=pd.DataFrame.from_dict(d,orient='index').T #length of utterance for each speaker

print(df)

      Kozinskey  PM-ME-WORRIES  ...  rhisoneros  crash_over-ride
0          58.0            1.0  ...        17.0              1.0
1          42.0            8.0  ...        49.0              3.0
2          42.0           19.0  ...         9.0             21.0
3          33.0            5.0  ...        96.0              7.0
4          14.0           47.0  ...        90.0             55.0
...         ...            ...  ...         ...              ...
8198        NaN            NaN  ...         NaN              NaN
8199        NaN            NaN  ...         NaN              NaN
8200        NaN            NaN  ...         NaN              NaN
8201        NaN            NaN  ...         NaN              NaN
8202        NaN            NaN  ...         NaN              NaN

[8203 rows x 13 columns]


In [ ]:
# code minimally adapted from Doyle et al, 2016 / Yurovski et al., 2016

import operator, itertools
def group(utterances):
	utterances.sort(key=operator.itemgetter('convId'))
	list1 = []
	for key, items in itertools.groupby(utterances, operator.itemgetter('convId')):
		list1.append(list(items))
  
	return list1


def allMarkers(markers):
	categories = []
	for marker in markers:
		categories.append(marker)
	return categories

def checkMarkers(markers):
	toReturn = []
	for marker in markers:
		if isinstance(marker, str):
			toReturn.append({"marker": marker, "category": marker})
		else:
			toReturn.append(marker)
	return toReturn


def findMarkersInConvo(markers,convo):
	ba = {} # Number of times Person A and person B says the marker["marker"]
	bna = {}
	nbna = {}
	nba = {}
	for utterance in convo:				
		for j, marker in enumerate(markers):
			word = marker["marker"]
			msgMarker = word in utterance["msgMarkers"]
			replyMarker = word in utterance["replyMarkers"]
			
			if msgMarker and replyMarker:
				ba[word] = ba.get(word,0) + 1
			elif replyMarker and not msgMarker:
				bna[word] = bna.get(word,0) + 1
			elif not replyMarker and msgMarker:
				nba[word] = nba.get(word,0) + 1
			else:
				nbna[word] = nbna.get(word,0) + 1
	#		print(msgMarker, replyMarker)
			print(ba, bna, nba, nbna)
	return({'ba': ba,'bna': bna,'nba': nba,'nbna': nbna}) 
 
def metaDataExtractor(groupedUtterances, markers,corpusType=''):
	results = []
	for i, convo in enumerate(groupedUtterances):
				
		toAppend = findMarkersInConvo(markers,convo)		
		results.append(toAppend)
	return results

def readMarkers(markersFile,dialect=None):
	if dialect is None:
		reader = csv.reader(open(markersFile))
	else:
		reader = csv.reader(open(markersFile),dialect=dialect)
	markers = []
	print('marker\tcategory')
	for i, row in enumerate(reader):
		toAppend = {}
		toAppend["marker"] = row[0]
		if(len(row) > 1):
			toAppend["category"] = row[1]
		else:
			toAppend["category"] = row[0]
		markers.append(toAppend)
		#print(toAppend["marker"]+'\t'+toAppend["category"])
	return markers

def writeFile(results, outputFile, shouldWriteHeader):
	if len(results) == 0:
		print("x")
		return
	toWrite = []
	header = sorted(list(results[0].keys()))
	for row in results:
		toAppend = []
		for key in header:
			toAppend.append(row[key])
		toWrite.append(toAppend)
	if shouldWriteHeader:
		with open(outputFile, "w", newline='') as f:
			writer = csv.writer(f)
			writer.writerows([header])
		f.close()
	with open(outputFile, "a", newline='') as f:
		writer = csv.writer(f)
		writer.writerows(toWrite)
	f.close()

def determineCategories(msgMarkers,catdict,useREs=False):
	msgCats = []
	#iterate over catdict items {category: [words/REs]}
	for cd in catdict.items():
		if useREs:
			if any(any(wordre.match(marker) for marker in msgMarkers) for wordre in cd[1]):	#if REs, see if any tokens match each RE
				msgCats.append(cd[0])
		else:
			if any(word in msgMarkers for word in cd[1]):			#if just words, see if any word in category also in msg
				msgCats.append(cd[0])
	return msgCats

def makeCatDict(markers,useREs=False):
	mdict = {}
	for m in markers:
		marker = re.compile(''.join([m["marker"], '$'])) if useREs else m["marker"]
		if m["category"] in mdict:
			mdict[m["category"]].append(marker)
		else:
			mdict[m["category"]] = [marker]
		#mdict[m["category"]] = mdict.get(m["category"],[]).append(m["marker"])	#Need to swap marker and category labels
		#mdict[m["marker"]] = mdict.get(m["marker"],[]).append(m["category"])
	return(mdict)
 
def createAlignmentDict(category,result,smoothing,corpusType=''):
	toAppend = {}
	print(category)
	print("R", result)
	ba = int(result["ba"].get(category, 0))
	bna = int(result["bna"].get(category, 0))
	nbna = int(result["nbna"].get(category, 0))
	nba = int(result["nba"].get(category, 0))
	print(ba,bna,nbna,nba)
	#Calculating alignment only makes sense if we've seen messages with and without the marker
	if (((ba+nba)==0 or (bna+nbna)==0)):
		return(None)
	
	toAppend["category"] = category
		
	#Calculating Echoes of Power alignment 
	powerNum = ba
	powerDenom = ba+nba
	baseNum = ba+bna
	baseDenom = ba+nba+bna+nbna

	if(powerDenom != 0 and baseDenom != 0):
		dnmalignment = powerNum/powerDenom - baseNum/baseDenom
		toAppend["dnmalignment"] = dnmalignment
	else:
		toAppend["dnmalignment"] = False
	
	powerNum = ba
	powerDenom = ba+nba
	baseDenom = bna+nbna
	baseNum = bna
	powerProb = math.log((powerNum+smoothing)/float(powerDenom+2*smoothing))
	baseProb = math.log((baseNum+smoothing)/float(baseDenom+2*smoothing))
	alignment = powerProb - baseProb
	toAppend["alignment"] = alignment
	
	toAppend["ba"] = ba
	toAppend["bna"] = bna
	toAppend["nba"] = nba
	toAppend["nbna"] = nbna
	#print("toapp:", toAppend)
	return(toAppend)
 
def runFormula(results, markers, smoothing,corpusType):
	toReturn = []
	categories = allMarkers(markers)
	#print("h", results) 
	#print("c", categories)
	for i, result in enumerate(results):
		for j, category in enumerate(categories):
			toAppend = createAlignmentDict(category["category"],result,smoothing,corpusType)
		#	print("here", toAppend)   
			if toAppend is not None:
				toReturn.append(toAppend)
	toReturn = sorted(toReturn, key=lambda k: (k["speakerId"],k["replierId"],k["category"]))
	return toReturn

In [ ]:
import pprint

# pair utterances from convokit 
markers = ["Hufflepuff", "Cornell", "Rowling", "absolutely", "Magic", "Howgarts", "the", "of", "very", "to", "through", "and","each", "other", "'d", "more", "less", "front", "back", "please", "any", "due", "just", "a", "in", "that", "for", "on", "are", "you", "it", "was", "were", "I", "as", "with", "they", "be", "at", "too", "have", "does", "this", "from", "or", "had", "by", "but", "some", "also", "what", "can", "out", "all", "your", "my", "up", "even", "so", "yes", "when", "almost", "no", "must", "should", "will", "would", "not"]


utterances =[]
previous_utt = dict()
#convo = corpus.random_conversation()

convo_utts= list(convo.iter_utterances()) 


for i in convo.traverse('bfs'): #iterate through utterances in tree-like (breadth) structure, level by level
  d = {} #each pair of speakers is a 'd' dictionary
  previous_utt[i.id] = (i.text,i.speaker.id)
  if i.reply_to == None:  #ignore if no reply
    continue
  d['convId'] = (previous_utt[i.reply_to][1], i.speaker.id)
  d['speakerId'] = previous_utt[i.reply_to][1] #?
  d['msg'] = previous_utt[i.reply_to][0]
  d['reply'] = i.text
  d['msgMarkers'] = []
  d['replyMarkers'] = []
  d['msgTokens']= []
  d['replyTokens']= []
  d["replierId"] = i.speaker.id
 # pprint.pprint(d)
  utterances.append(d)


for idx, utt in enumerate(utterances):  #append marker in utterance metadata if it exists
  for marker in markers:
			if marker in utt['msg']:
				utterances[idx]['msgMarkers'].append(marker)
			if marker in utt['reply']:
				utterances[idx]['replyMarkers'].append(marker)




In [ ]:
pprint.pprint(utterances)

[{'convId': ('Darth_Saruman9', 'StrykerCosimoStorm'),
  'msg': '**Due to overwhelming popular demand (shoutout to Regulus Arterus '
         'Black), Aragorn Gimli and Legolas are back with more invaluably '
         'inciteful commentary on mapping out Cornell Campus and collegetown. '
         'As ever, we appreciate your commentary and look forward to '
         'thoughtful discussion. The topic today will be J.K. Rowling’s '
         'Wizarding Universe as locations around Cornell. Please keep your '
         'comments in the discussion respectful to all members of our '
         'community and refrain from any inappropriate or derogatory language. '
         'Additionally, try and support your arguments with textual evidence '
         'and proper citations. Looking forward to getting some ideas flowing. '
         'Here are our preliminary rankings heading into 2019 rush:**   \n'
         '\n'
         '\n'
         '**Donlon - Gyfindor Tower**  \n'
         '**CKB - Slytherin du

In [ ]:
#main function - run alignment

smoothing=1
shouldWriteHeader=True
outputFile='aaaaaaaaa'
markers = checkMarkers(markers)
import math, csv
groupedUtterances = group(utterances)
#print(groupedUtterances)
metaData = metaDataExtractor(groupedUtterances,markers) 
#print(metaData)        
results = runFormula(metaData, markers, smoothing,'')
writeFile(results, outputFile, shouldWriteHeader)


{} {} {} {'Hufflepuff': 1}
{} {} {} {'Hufflepuff': 1, 'Cornell': 1}
{} {} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1}
{} {} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1}
{} {} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1}
{} {} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1, 'Howgarts': 1}
{} {'the': 1} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1, 'Howgarts': 1}
{} {'the': 1} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1, 'Howgarts': 1, 'of': 1}
{} {'the': 1} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1, 'Howgarts': 1, 'of': 1, 'very': 1}
{} {'the': 1} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1, 'Howgarts': 1, 'of': 1, 'very': 1, 'to': 1}
{} {'the': 1} {} {'Hufflepuff': 1, 'Cornell': 1, 'Rowling': 1, 'absolutely': 1, 'Magic': 1, 'Howgarts': 1, 'of': 1, 'very': 1, 'to': 1, 